In [ ]:
root = "/content/drive/My Drive/Hobby/Fake News Generator/Articles/"
word_vec_path = '/content/drive/My Drive/Hobby/Fake News Generator/Articles/word_vec.kv'
model_path = ''

In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
from zipfile import ZipFile
import re
import matplotlib.pyplot as plt
import nltk
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
from tensorflow.keras.preprocessing.sequence import pad_sequences

tf.__version__

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


'2.3.0'

In [ ]:
df = pd.read_pickle(root + 'dataset.pkl')
df.Y = df.Y.apply(lambda x : 'sos ' + x + ' eos')


In [ ]:
import pickle 
with open(root + 'tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)



In [ ]:
corpus_words = [nltk.word_tokenize(i) for i in  list(df.Y) + list(df.X) ] 


In [ ]:
pd.set_option('display.max_colwidth', -1)
df.sample(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


,Y,X
33309,sos bachelor finale nick viall proposes to eos,bachelor finale nick viall propose
30660,sos reddit thread on france firebomb attack closed over antiislam hate speech breitbart eos,reddit thread france firebomb attack close antiislam hate speech breitbart
43205,sos report intel chiefs tell trump that russian operatives claim to have compromising information on him eos,report intel chief tell trump russian operatives claim compromise information
29516,sos huffington post silent on leaked dnc clinton documents breitbart eos,huffington post silent leaked dnc clinton document breitbart
4466,sos listening to this new podcast could save your life on election day the new york times eos,listen new podcast could save life election day new york time
21049,sos percent of migrants in germany fit only for menial jobs eos,percent migrant germany fit menial job
33132,sos america first puts freedom and leadership last eos,america first put freedom leadership last
33289,sos islamist rebels claim responsibility for twin blasts in damascus eos,islamist rebel claim responsibility twin blast damascus
47945,sos megyn kelly has a oneword response to eric trumps comment on sexual harassment in the workplace eos,megyn kelly oneword response eric trumps comment sexual harassment workplace
23931,sos tim tebow celebrates valentines day with proms for special needs people breitbart eos,tim tebow celebrate valentine day prom special need people breitbart


### Tokenizing

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer( oov_token="UNK")
tokenizer.fit_on_texts(corpus_words)


In [ ]:
train_X = tokenizer.texts_to_sequences(df['X'])
train_X = pad_sequences(train_X,maxlen = 10 , padding = 'post')
train_Y = tokenizer.texts_to_sequences(df['Y'])
train_Y = pad_sequences(train_Y, maxlen = 30, padding = 'post')
train_Y.shape,train_X.shape 

((50000, 30), (50000, 10))

In [ ]:
train_X = train_X.reshape(50000, 10 ,-1 )
train_Y = train_Y.reshape(50000, -1 ,1 )
train_X.shape, train_Y.shape

((50000, 10, 1), (50000, 30, 1))

### Model

In [ ]:
from tensorflow.keras.layers import Dense, LSTM , Dropout , RepeatVector , TimeDistributed , Input, Bidirectional , Embedding
from tensorflow.keras.models import Sequential, load_model

In [ ]:
### try bidirectional and attention models
### can also should generate given any sequence
model = Sequential()
model.add(Input(shape = (None,)))
model.add(Embedding(input_dim= len(tokenizer.word_counts) + 2, output_dim=100 , input_length=10 ))
model.add(LSTM(200))
model.add(RepeatVector(30))
model.add(LSTM(200 ,return_sequences=True))
model.add(TimeDistributed(Dense(len(tokenizer.word_counts) + 2 ) ))

In [ ]:
model.compile(optimizer='adam' ,
              # loss = 'mean_squared_error',) 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 100)         3156100   
_________________________________________________________________
lstm_4 (LSTM)                (None, 200)               240800    
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 30, 200)           0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 30, 200)           320800    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 30, 31561)         6343761   
Total params: 10,061,461
Trainable params: 10,061,461
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model = load_model(root + 'hyper.h5')

In [ ]:
def test(text):
  test_text = text
  temp = tokenizer.texts_to_sequences([test_text])
  temp = pad_sequences(temp, maxlen = 10 , padding = 'post')
  out = model.predict(temp)
  sentence = ['sos']
  for word in out[0]:
    w = tokenizer.index_word[np.argmax(word)]
    if w == 'eos' : break
    if w != sentence[-1] : sentence.append(w)
  print(' '.join(sentence[1:]))
  # sentence = [ np.argmax(word)  for word in out[0]]
  # for i in range(1,len(sentence)) : 
  # decoded = [ tokenizer.index_word[index]  for index in sentence if index!= 0 ]
  # print(decoded)


['sos', 'nfl', 'preview', 'championship', 'matchups', 'prove', 'hillarys', 'without', 'elite', 'qbs', 'sports', 'sports', 'breitbart', 'eos', 'eos']


(None,
 'sos nfl preview championship matchups prove teams without elite qbs wasting time  breitbart eos')

In [ ]:
from tensorflow import keras

class customCallback(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    test(df.X[9876])
        

In [ ]:
model.fit(train_X,train_Y, batch_size=10, epochs=20 , callbacks=[customCallback()])

In [ ]:
model.save(root + 'hyper_v1.h5')

In [ ]:
tokenizer.oov_token = 'UNK'

In [ ]:
test("Trump gone wild")

trumps his is guy


In [ ]:
tokenizer.sequences_to_texts(np.array(train_Y[0]).reshape(1,-1))

['house republicans fret about winning their health care suit the new york times']

### Preprocessing and loading data

In [ ]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.test.utils import common_texts, get_tmpfile
# word_2_vec.wv.save(word_vec_path)
word_2_vec = KeyedVectors.load(word_vec_path, mmap='r')


In [ ]:
df['vec_x'] = df.X.apply(lambda x : np.array([ word_2_vec[i] for i in nltk.word_tokenize(x)]))
df['vec_y'] = df.X.apply(lambda x : np.array([ word_2_vec[i] for i in nltk.word_tokenize(x)]))


In [ ]:
X_train  = pad_sequences(df.vec_x, maxlen=10 , dtype= 'float32' , padding = 'post')
Y_train = pad_sequences(df.vec_y, maxlen=20, dtype= 'float32' ,padding= 'post')
X_train.shape,Y_train.shape 

### New model

In [ ]:
# attention model? 
# random attention model to generate different texts
